In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#读取路网数据
originRoad = gpd.read_file("../suzhou/data/原始数据/road/苏州.shp")
print(originRoad.shape)
originRoad = originRoad[['ID', 'KIND', 'WIDTH', 'DIRECTION', 'FUNCCLASS', 'LENGTH', 'SPEEDCLASS', 'PHYLANES2E', 'PHYLANEE2S', 'geometry']]
print(originRoad.shape)

In [8]:
originRoad[:100]

,ID,KIND,WIDTH,DIRECTION,FUNCCLASS,LENGTH,UFLAG,SPEEDCLASS,PHYLANES2E,PHYLANEE2S,ROADNAME,ROADNO,geometry
0,101194417,060a,55,2,5,0.179,0,5,2,0,花安路,None,"LINESTRING Z (121.15275 31.28852 0.00000, 121...."
1,101194420,0602,30,3,5,0.045,0,5,0,1,None,None,"LINESTRING Z (121.15253 31.28720 0.00000, 121...."
2,101194421,0602,30,3,5,0.011,0,5,0,1,None,None,"LINESTRING Z (121.15256 31.28760 0.00000, 121...."
3,101194422,080e|081a,55,1,5,0.038,0,7,1,1,None,None,"LINESTRING Z (121.15303 31.28761 0.00000, 121...."
4,101194423,0604,55,1,5,0.007,0,7,1,1,None,None,"LINESTRING Z (121.15263 31.28760 0.00000, 121...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15023346,0601,55,1,4,0.052,0,6,1,1,萧林东路,Ｙ３１２,"LINESTRING Z (121.01824 31.40599 0.00000, 121...."
96,15023350,0802|0814,30,3,5,0.093,0,7,0,1,亿宏大道,None,"LINESTRING Z (121.01638 31.40603 0.00000, 121...."
97,15023352,0802|0814,30,2,5,0.093,0,7,1,0,亿宏大道,None,"LINESTRING Z (121.01638 31.40603 0.00000, 121...."
98,15023354,0804|0814,30,1,5,0.008,0,7,1,1,广场路,None,"LINESTRING Z (121.01623 31.40685 0.00000, 121...."


In [2]:
#处理卡口数据
file202109 = "../suzhou/data/卡口数据/苏州卡口过车/卡口过车/2021年9月过车.txt"

df202109 = pd.read_csv(file202109,sep='\s+', header=None, names=['DeviceID', 'PlateColor', 'PassTimeHour', 'Quantity'])
print(df202109.shape)
#打印10000到10005行的数据
print(df202109.iloc[10000:10005])

C:\Anaconda\envs\CTVI\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (3,5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(14721113, 4)
                         DeviceID  PlateColor PassTimeHour Quantity
| 123456789123456789 | 6        |  1632751200            |       25
                       6        |  1632754800            |       16
                       6        |  1632758400            |        9
                       6        |  1632762000            |        8
                       6        |  1632765600            |        4


In [3]:
with open(file202109, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    line_count = len(lines)

# 打印行数
print("file202109:", line_count)

file202109: 14721113


In [12]:
from collections import defaultdict
from datetime import datetime
# 定义一个字典来存储每天的统计数据
# 结构：{日期: {DeviceID: 过车量}}
test_stats = defaultdict(lambda: defaultdict(int))
#测试文件处理准确性
filetest = "../suzhou/data/卡口数据/苏州卡口过车/卡口过车/text.txt"
with open(filetest, 'r', encoding='utf-8') as file:
    for i in range(6):
        # 读取一行
        line = file.readline()
        # 如果读到文件末尾，退出循环
        elements = line.split("|")
        # 提取需要的部分（去掉首尾的空格）
        device_id = elements[1].strip()
        if device_id == "":
            print("DeviceID is empty")
            continue
        plate_color = elements[2].strip()
        pass_time = elements[3].strip()
        quantity = elements[4].strip()
        timeStamp = int(pass_time)
        #将pass_time转换为时间戳
        dt_object = datetime.fromtimestamp(timeStamp)
        date_key = dt_object.strftime("%Y-%m-%d-%H")
        print(date_key)
        # 统计过车量
        test_stats[date_key][device_id] += int(quantity)

2021-09-29-09
2021-09-29-10
2021-09-29-11
2021-09-29-12
2021-09-29-13
2021-09-29-14


In [11]:
#遍历test_stats
for date, stats in test_stats.items():
    print(f"{date}:")
    for device_id, total_quantity in stats.items():
        print(f"  {device_id}: {total_quantity}")

2021-09-29-09:
  1: 3
2021-09-29-10:
  1: 8
2021-09-29-11:
  2: 13
2021-09-29-12:
  3: 8
2021-09-29-13:
  3: 9
2021-09-29-14:
  4: 16


In [5]:
# 日期格式测试，转换为年-月-日-时
from datetime import datetime
with open(file202109, 'r', encoding='utf-8') as file:
    for i in range(14721113):
        line = file.readline()
        # 如果读到文件末尾，退出循环
        elements = line.split("|")
        # 提取需要的部分（去掉首尾的空格）
        device_id = elements[1].strip()
        if device_id == "":
            print("DeviceID is empty")
            #continue
        plate_color = elements[2].strip()
        pass_time = elements[3].strip()
        quantity = elements[4].strip()
        timeStamp = int(pass_time)
        #将pass_time转换为时间戳
        dt_object = datetime.fromtimestamp(timeStamp)
        date_key = dt_object.strftime("%Y-%m-%d-%H")
        print(date_key)
        if i>20:
            break

DeviceID is empty
2021-09-06-09
DeviceID is empty
2021-09-07-12
DeviceID is empty
2021-09-23-14
DeviceID is empty
2021-09-26-14
DeviceID is empty
2021-09-29-11
DeviceID is empty
2021-09-29-17
DeviceID is empty
2021-09-30-08
DeviceID is empty
2021-09-30-10
DeviceID is empty
2021-09-01-00
DeviceID is empty
2021-09-01-07
DeviceID is empty
2021-09-01-09
DeviceID is empty
2021-09-01-10
DeviceID is empty
2021-09-01-11
DeviceID is empty
2021-09-01-15
DeviceID is empty
2021-09-01-16
DeviceID is empty
2021-09-01-18
DeviceID is empty
2021-09-01-20
DeviceID is empty
2021-09-01-21
DeviceID is empty
2021-09-02-01
DeviceID is empty
2021-09-02-02
DeviceID is empty
2021-09-02-07
DeviceID is empty
2021-09-02-08


In [6]:
from collections import defaultdict
# 定义一个字典来存储每天的统计数据
# 结构：{日期: {DeviceID: 过车量}}
daily_stats = defaultdict(lambda: defaultdict(int))

In [7]:
from datetime import datetime
import csv
#根据deviceID提取需要的部分
with open(file202109, 'r', encoding='utf-8') as file:
    for i in range(14721113):
        # 读取一行
        line = file.readline()
        # 如果读到文件末尾，退出循环
        elements = line.split("|")
        # 提取需要的部分（去掉首尾的空格）
        device_id = elements[1].strip()
        if device_id == "":
            print("DeviceID is empty")
            continue
        plate_color = elements[2].strip()
        pass_time = elements[3].strip()
        quantity = elements[4].strip()
        timeStamp = int(pass_time)
        #将pass_time转换为时间戳
        dt_object = datetime.fromtimestamp(timeStamp)
        date_key = dt_object.strftime("%Y-%m-%d-%H")
        # 统计过车量
        daily_stats[date_key][device_id] += int(quantity)
        # 测试：打印前 10 行
        #if i < 10:
        #    print(f"DeviceID: {device_id}, Date: {date_key}, Quantity: {quantity}")

DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID i

In [8]:
import os
import csv
# 遍历 daily_stats
for date, stats in daily_stats.items():
    # 解析日期字符串
    year, month, day, hour = date.split('-')
    # 创建文件夹路径
    folder_path = os.path.join(year, f"{month}-{day}")
    # 确保文件夹存在
    os.makedirs(folder_path, exist_ok=True)
    # 生成文件名，例如 12.csv
    filename = f"{hour}.csv"
    file_path = os.path.join(folder_path, filename)
    # 写入数据到 CSV 文件
    with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        # 写入表头
        writer.writerow(["DeviceID", "Quantity"])
        # 写入数据
        for device_id, total_quantity in stats.items():
            writer.writerow([device_id, total_quantity])

    print(f"文件 {file_path} 已生成")

文件 2021\09-03\22.csv 已生成
文件 2021\09-12\12.csv 已生成
文件 2021\09-18\18.csv 已生成
文件 2021\09-21\18.csv 已生成
文件 2021\09-21\19.csv 已生成
文件 2021\09-24\18.csv 已生成
文件 2021\09-24\19.csv 已生成
文件 2021\09-26\08.csv 已生成
文件 2021\09-26\18.csv 已生成
文件 2021\09-02\07.csv 已生成
文件 2021\09-02\08.csv 已生成
文件 2021\09-02\09.csv 已生成
文件 2021\09-02\12.csv 已生成
文件 2021\09-02\16.csv 已生成
文件 2021\09-02\17.csv 已生成
文件 2021\09-03\07.csv 已生成
文件 2021\09-03\08.csv 已生成
文件 2021\09-03\09.csv 已生成
文件 2021\09-03\13.csv 已生成
文件 2021\09-03\14.csv 已生成
文件 2021\09-04\08.csv 已生成
文件 2021\09-04\09.csv 已生成
文件 2021\09-04\14.csv 已生成
文件 2021\09-05\09.csv 已生成
文件 2021\09-05\10.csv 已生成
文件 2021\09-05\12.csv 已生成
文件 2021\09-05\15.csv 已生成
文件 2021\09-06\08.csv 已生成
文件 2021\09-06\09.csv 已生成
文件 2021\09-06\16.csv 已生成
文件 2021\09-06\17.csv 已生成
文件 2021\09-07\07.csv 已生成
文件 2021\09-07\08.csv 已生成
文件 2021\09-07\09.csv 已生成
文件 2021\09-07\11.csv 已生成
文件 2021\09-07\12.csv 已生成
文件 2021\09-07\15.csv 已生成
文件 2021\09-07\16.csv 已生成
文件 2021\09-08\07.csv 已生成
文件 2021\09-08\08.csv 已生成


In [29]:
testfile = "../suzhou/data/卡口数据/苏州卡口过车/卡口过车/text.txt"
testDict = defaultdict(lambda: defaultdict(int))
from datetime import datetime
import csv
#根据deviceID提取需要的部分
with open(testfile, 'r', encoding='utf-8') as file:
    for i in range(6):
        # 读取一行
        line = file.readline()
        # 如果读到文件末尾，退出循环
        elements = line.split("|")
        # 提取需要的部分（去掉首尾的空格）
        device_id = elements[1].strip()
        if device_id == "":
            print("DeviceID is empty")
            continue
        plate_color = elements[2].strip()
        pass_time = elements[3].strip()
        quantity = elements[4].strip()
        timeStamp = int(pass_time)
        #将pass_time转换为时间戳
        dt_object = datetime.fromtimestamp(timeStamp)
        date_key = dt_object.strftime("%Y-%m-%d")  # 按天分组
        # 统计过车量
        testDict[date_key][device_id] += int(quantity)

# 将统计结果保存到不同的文件中
for date, stats in testDict.items():
    # 生成文件名，例如 2021-9-1.csv
    filename = f"{date}.csv"
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        # 写入表头
        writer.writerow(["DeviceID", "Quantity"])
        # 写入数据
        for device_id, total_quantity in stats.items():
            writer.writerow([device_id, total_quantity])

    print(f"文件 {filename} 已生成")

文件 2021-09-29.csv 已生成


In [14]:
file202112 = "../suzhou/data/卡口数据/苏州卡口过车/卡口过车/2021年12月过车.txt"
with open(file202112, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    line_count = len(lines)

# 打印行数
print("file202112:", line_count)

file202112: 15936800


In [15]:
file202112Dict = defaultdict(lambda: defaultdict(int))
from datetime import datetime
import csv
#根据deviceID提取需要的部分
with open(file202112, 'r', encoding='utf-8') as file:
    for i in range(15936799):
        # 读取一行
        line = file.readline()
        # 如果读到文件末尾，退出循环
        elements = line.split("|")
        # 提取需要的部分（去掉首尾的空格）
        device_id = elements[1].strip()
        if device_id == "":
            print("DeviceID is empty")
            continue
        plate_color = elements[2].strip()
        pass_time = elements[3].strip()
        quantity = elements[4].strip()
        timeStamp = int(pass_time)
        #将pass_time转换为时间戳
        dt_object = datetime.fromtimestamp(timeStamp)
        date_key = dt_object.strftime("%Y-%m-%d-%H")
        # 统计过车量
        file202112Dict[date_key][device_id] += int(quantity)

# 将统计结果保存到不同的文件中
for date, stats in file202112Dict.items():
    # 生成文件名，例如 2021-9-1.csv
    year, month, day, hour = date.split('-')
    # 创建文件夹路径
    folder_path = os.path.join(year, f"{month}-{day}")
    # 确保文件夹存在
    os.makedirs(folder_path, exist_ok=True)
    # 生成文件名，例如 12.csv
    filename = f"{hour}.csv"
    file_path = os.path.join(folder_path, filename)
    # 写入数据到 CSV 文件
    with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        # 写入表头
        writer.writerow(["DeviceID", "Quantity"])
        # 写入数据
        for device_id, total_quantity in stats.items():
            writer.writerow([device_id, total_quantity])

    print(f"文件 {file_path} 已生成")

DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID i

In [16]:
file202203 = "../suzhou/data/卡口数据/苏州卡口过车/卡口过车/2022年3月过车.txt"
with open(file202203, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    line_count = len(lines)

# 打印行数
print("file202203:", line_count)
file202203Dict = defaultdict(lambda: defaultdict(int))

file202203: 15086788


In [17]:
from datetime import datetime
import csv
#根据deviceID提取需要的部分
with open(file202203, 'r', encoding='utf-8') as file:
    for i in range(15086787):
        # 读取一行
        line = file.readline()
        # 如果读到文件末尾，退出循环
        elements = line.split("|")
        # 提取需要的部分（去掉首尾的空格）
        device_id = elements[1].strip()
        if device_id == "":
            print("DeviceID is empty")
            continue
        plate_color = elements[2].strip()
        pass_time = elements[3].strip()
        quantity = elements[4].strip()
        timeStamp = int(pass_time)
        #将pass_time转换为时间戳
        dt_object = datetime.fromtimestamp(timeStamp)
        date_key = dt_object.strftime("%Y-%m-%d-%H")  # 按天分组
        # 统计过车量
        file202203Dict[date_key][device_id] += int(quantity)

# 将统计结果保存到不同的文件中
for date, stats in file202203Dict.items():
    year, month, day, hour = date.split('-')
    # 创建文件夹路径
    folder_path = os.path.join(year, f"{month}-{day}")
    # 确保文件夹存在
    os.makedirs(folder_path, exist_ok=True)
    # 生成文件名，例如 12.csv
    filename = f"{hour}.csv"
    file_path = os.path.join(folder_path, filename)
    # 写入数据到 CSV 文件
    with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        # 写入表头
        writer.writerow(["DeviceID", "Quantity"])
        # 写入数据
        for device_id, total_quantity in stats.items():
            writer.writerow([device_id, total_quantity])

    print(f"文件 {file_path} 已生成")

DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID i

In [18]:
file202206 = "../suzhou/data/卡口数据/苏州卡口过车/卡口过车/2022年6月过车.txt"
with open(file202206, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    line_count = len(lines)

# 打印行数
print("file202206:", line_count)

file202206: 15207665


In [19]:
file202206Dict = defaultdict(lambda: defaultdict(int))
from datetime import datetime
import csv
#根据deviceID提取需要的部分
with open(file202206, 'r', encoding='utf-8') as file:
    for i in range(15207665):
        # 读取一行
        line = file.readline()
        # 如果读到文件末尾，退出循环
        elements = line.split("|")
        # 提取需要的部分（去掉首尾的空格）
        device_id = elements[1].strip()
        if device_id == "":
            print("DeviceID is empty")
            continue
        plate_color = elements[2].strip()
        pass_time = elements[3].strip()
        quantity = elements[4].strip()
        timeStamp = int(pass_time)
        #将pass_time转换为时间戳
        dt_object = datetime.fromtimestamp(timeStamp)
        date_key = dt_object.strftime("%Y-%m-%d-%H")  # 按天分组
        # 统计过车量
        file202206Dict[date_key][device_id] += int(quantity)

# 将统计结果保存到不同的文件中
for date, stats in file202206Dict.items():
    year, month, day, hour = date.split('-')
    # 创建文件夹路径
    folder_path = os.path.join(year, f"{month}-{day}")
    # 确保文件夹存在
    os.makedirs(folder_path, exist_ok=True)
    # 生成文件名，例如 12.csv
    filename = f"{hour}.csv"
    file_path = os.path.join(folder_path, filename)
    # 写入数据到 CSV 文件
    with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        # 写入表头
        writer.writerow(["DeviceID", "Quantity"])
        # 写入数据
        for device_id, total_quantity in stats.items():
            writer.writerow([device_id, total_quantity])

    print(f"文件 {file_path} 已生成")

DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID is empty
DeviceID i